<a href="https://colab.research.google.com/github/ansonkwokth/TableTennisPrediction/blob/main/inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
!git clone https://github.com/ansonkwokth/TableTennisPrediction.git
%cd TableTennisPrediction

Cloning into 'TableTennisPrediction'...
remote: Enumerating objects: 277, done.
remote: Counting objects: 100% (116/116), done.
remote: Compressing objects: 100% (101/101), done.
remote: Total 277 (delta 70), reused 15 (delta 15), pack-reused 161 (from 1)
Receiving objects: 100% (277/277), 3.90 MiB | 9.78 MiB/s, done.
Resolving deltas: 100% (138/138), done.
/content/TableTennisPrediction/TableTennisPrediction


In [74]:

import pandas as pd
from tabulate import tabulate
from utils import data_loader as dl

import numpy as np
from model.Elo import Elo
from model.ModifiedElo import ModifiedElo
from model.ensemble import BaggingRatingSystem

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm

import copy

import warnings
warnings.filterwarnings('ignore')

In [75]:
# GAME = 'TTStar'
# GAME = 'TTCup'
# GAME = 'SetkaCup'
GAME = 'SetkaCupWomen'
# GAME = 'LigaPro'


In [76]:
match GAME:
    case 'TTStar':
        years = [2020, 2021, 2022, 2023, 2024]
    case 'TTCup':
        years = [2020, 2021, 2022, 2023, 2024]
    case 'SetkaCup':
        years = [2020, 2021, 2022, 2023, 2024]
    case 'SetkaCupWomen':
        years = [2020, 2021, 2022, 2023, 2024]
    case 'LigaPro':
        years = [2022, 2023, 2024]
    case _:
        raise ValueError("Invalid game selected.")


text_data_game = dl.load_game_data(GAME, years, '../')
text_data = {
    year: text_data_game[year] for year in years
}
df = dl.create_game_dfs(GAME, years, text_data)

Loading ..//SetkaCupWomen2020.txt
Loading ..//SetkaCupWomen2021.txt
Loading ..//SetkaCupWomen2022.txt
Loading ..//SetkaCupWomen2023.txt
Loading ..//SetkaCupWomen2024.txt


In [77]:
# Generate ID indices for each pair of rows in the DataFrame
idx_lt = [i for i in range(len(df) // 2) for _ in range(2)]
df['ID'] = idx_lt  # Assign to the 'ID' column

# Reset the DataFrame index to ensure it's sequential
df.reset_index(drop=True, inplace=True)

# Get unique players and store them in player_lt
player_lt = df['Player'].unique()



In [78]:
def format_to_array(df: pd.DataFrame) -> np.ndarray:

    # info_col = ['ID', 'Round', 'Datetime', 'Game', 'Date', 'Time']
    info_col = ['Round', 'Datetime', 'Game', 'Date', 'Time']
    col = [item for item in df.columns if item not in info_col]

    df[[c for c in col if "Set" in c]] = df[[c for c in col if "Set" in c]].astype(float)
    X = df[col].values.reshape(-1, 2, len(col))
    return X

In [79]:
X_all = format_to_array(df)

In [80]:
modelMElo = ModifiedElo()
for _ in range(5):
    modelMElo.fit(X_all)


Training model: 100%|██████████| 34579/34579 [00:01<00:00, 18014.04it/s]


# Print

In [81]:
def find_player(name):
    p_lt = []
    for k, v in modelMElo.params.items():
        if name in k:
            print(k)
            p_lt.append(k)
    return p_lt

In [82]:
def find_OU(player1, player2):
    set1_dt = modelMElo.predict_set_config(player1, player2)[-1]
    set2_dt = modelMElo.predict_set_config(player2, player1)[-1]

    set_sum_dt = {}
    for k1, v1 in set1_dt.items():
        v2 = set2_dt[k1]
        set_sum_dt[k1] = v1 + v2



    p_O = 0
    p_U = 0
    for k, v in set_sum_dt.items():
        if k > line:
            p_O += v
        else:
            p_U += v
    return p_O, p_U

In [106]:
df_lt = []

In [232]:
p1_lt = find_player("Volg")
print("-"*30)
p2_lt = find_player("Ivch")


Volgina A.
------------------------------
Ivchenko O.


In [233]:
odd1 = 2.50
odd2 = 1.50

In [234]:
player1 = p1_lt[0]
player2 = p2_lt[0]
player1, player2

('Volgina A.', 'Ivchenko O.')

In [235]:
time = "22:35"

date = "Feb 8"


line = 18.5


found_p1, found_p2, p_game = modelMElo.predict_game(player1, player2)
found_p1, found_p2, p_set = modelMElo.predict_set(player1, player2)
p_O, p_U = find_OU(player1, player2)
winner = player1 if p_game > 0.5 else player2
p_set_win = p_set if winner == player1 else 1 - p_set
p_game_win = p_game if winner == player1 else 1 - p_game
p_OU = p_O if p_O > p_U else p_U
OU = "O" if p_O > p_U else "U"

winnerOdd = odd1 if winner == player1 else odd2
loserOdd = odd2 if winner == player1 else odd1

In [236]:
if (found_p1 and found_p2):
    df_row = pd.DataFrame({
                         "Time": [time],
                         "Player1": [player1], "Player2": [player2], "Winner": [winner],
                         "Set": [f"{p_set_win*100:.1f}%"], "Game": [f"{p_game_win*100:.1f}%"],
                         "pGame": [p_game_win],
                         "WinnerOdd": [winnerOdd], "LoserOdd": [loserOdd]
                         })
                        #  "O/U": [f"{OU} {line}: {p_OU*100:.1f}%"]})

    print(tabulate(df_row, headers = 'keys', tablefmt = 'psql', showindex=False))


+--------+------------+-------------+-------------+-------+--------+----------+-------------+------------+
| Time   | Player1    | Player2     | Winner      | Set   | Game   |    pGame |   WinnerOdd |   LoserOdd |
|--------+------------+-------------+-------------+-------+--------+----------+-------------+------------|
| 22:35  | Volgina A. | Ivchenko O. | Ivchenko O. | 55.5% | 60.2%  | 0.602157 |         1.5 |        2.5 |
+--------+------------+-------------+-------------+-------+--------+----------+-------------+------------+


In [237]:
p_U, p_O

(0.4839102277565401, 0.5160897722434594)

In [238]:
df_lt.append(df_row)

In [239]:
df_print = pd.concat(df_lt)
df_print.index = range(len(df_lt))
print(tabulate(df_print, headers = 'keys', tablefmt = 'psql', showindex=False))
# print(tabulate(df_print, headers = 'keys', tablefmt = 'psql'))

+--------+-------------+-------------+-------------+-------+--------+----------+-------------+------------+
| Time   | Player1     | Player2     | Winner      | Set   | Game   |    pGame |   WinnerOdd |   LoserOdd |
|--------+-------------+-------------+-------------+-------+--------+----------+-------------+------------|
| 15:35  | Lapa H.     | Gordeets M. | Gordeets M. | 59.6% | 67.5%  | 0.675366 |        1.16 |       4.5  |
| 16:05  | Severina Y. | Ivchenko O. | Ivchenko O. | 65.9% | 77.9%  | 0.778758 |        1.33 |       3.25 |
| 16:35  | Man A.      | Volgina A.  | Man A.      | 58.3% | 65.2%  | 0.652275 |        1.66 |       2.1  |
| 17:05  | Lapa H.     | Ivchenko O. | Ivchenko O. | 61.1% | 70.1%  | 0.701008 |        1.14 |       5    |
| 17:35  | Volgina A.  | Gordeets M. | Gordeets M. | 53.1% | 55.9%  | 0.558541 |        1.5  |       2.5  |
| 18:05  | Severina Y. | Man A.      | Man A.      | 68.8% | 82.1%  | 0.821286 |        1.3  |       3.4  |
| 18:35  | Lapa H.     | Vol

In [240]:
df_print.to_csv("../df_print.csv")